In [13]:
# import libraries
import requests
import json
import pandas as pd
import numpy as np
import os
import datetime
from dotenv import load_dotenv
load_dotenv()


False

In [4]:
from jobs_scraper import JobsScraper

In [15]:

# Let's create a new JobsScraper object and perform the scraping for a given query.
scraper = JobsScraper(country="nl", position="Data Engineer", location="Amsterdam", pages=3)
df = scraper.scrape()
print('false')

Scraping in progress...: 100%|██████████| 3/3 [00:00<00:00, 17.87it/s]

false


In [12]:
# import module
import requests
from bs4 import BeautifulSoup
  
  
# user define function
# Scrape the data
# and get in string
def getdata(url):
    r = requests.get(url)
    return r.text
  
# Get Html code using parse
def html_code(url):
  
    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')
  
    # return html code
    return(soup)
  
# filter job data using
# find_all function
def job_data(soup):
    
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""
    for item in soup.find_all("a", class_="jobtitle turnstileLink"):
        data_str = data_str + item.get_text()
    result_1 = data_str.split("\n")
    return(result_1)
  
# filter company_data using
# find_all function
  
  
def company_data(soup):
  
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""
    result = ""
    for item in soup.find_all("div", class_="sjcl"):
        data_str = data_str + item.get_text()
    result_1 = data_str.split("\n")
  
    res = []
    for i in range(1, len(result_1)):
        if len(result_1[i]) > 1:
            res.append(result_1[i])
    return(res)
  
  
# driver nodes/main function
if __name__ == "__main__":
  
    # Data for URL
    job = "data+science+internship"
    Location = "Noida%2C+Uttar+Pradesh"
    url = "https://in.indeed.com/jobs?q="+job+"&l="+Location
  
    # Pass this URL into the soup
    # which will return
    # html string
    soup = html_code(url)
  
    # call job and company data
    # and store into it var
    job_res = job_data(soup)
    com_res = company_data(soup)
  
    # Traverse the both data
    temp = 0
    for i in range(1, len(job_res)):
        j = temp
        for j in range(temp, 2+temp):
            print("Company Name and Address : " + com_res[j])
  
        temp = j
        print("Job : " + job_res[i])
        print("-----------------------------")

In [16]:
def find_jobs_from(website, job_title, location, desired_characs, filename= "test.csv"):    
    """
    This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
    The arguments it takes are:
        - Website: to specify which website to search (options: 'Indeed' or 'CWjobs')
        - Job_title
        - Location
        - Desired_characs: this is a list of the job characteristics of interest,
            from titles, companies, links and date_listed.
        - Filename: to specify the filename and format of the output.
            Default is .xls file called 'results.xls'
    """
    
    if website == 'Indeed':
        job_soup = load_indeed_jobs_div(job_title, location)
        jobs_list, num_listings = extract_job_information_indeed(job_soup, desired_characs)
    
    if website == 'CWjobs':
        # TO DO LATER
    
        save_jobs_to_excel(jobs_list, filename)
 
def extract_job_information_indeed(job_soup, desired_characs):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
    cols = []
    extracted_info = []
    
    
    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem))
        extracted_info.append(titles)                    
    
    if 'companies' in desired_characs:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_indeed(job_elem))
        extracted_info.append(companies)
    
    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeed(job_elem))
        extracted_info.append(links)
    
    if 'date_listed' in desired_characs:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_indeed(job_elem))
        extracted_info.append(dates)
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings
    print('{} new job postings retrieved. Stored in {}.'.format(num_listings, filename))

In [6]:
from flask import Flask, render_template, request, send_file
from bs4 import BeautifulSoup
import requests
import csv

def get_so_jobs(term):
  job_list = []
  url = f"https://stackoverflow.com/jobs?r=true&q={term}"
  html_doc = requests.get(url).text
  soup = BeautifulSoup(html_doc, "html.parser")
  list_results = soup.find("div", {"class":"listResults"})
  head_soup = list_results.find_all("h2", {"class":"mb4 fc-black-800 fs-body3"})
  for head in head_soup:
    header = head.find("a")
    link = "https://stackoverflow.com" + header.get("href")
    title = header.get_text()
    job_list.append([title, "company_name", link])
  comp_soup = list_results.find_all("h3", {"class":"fc-black-700 fs-body1 mb4"})
  for i, company in enumerate(comp_soup):
    comp_name = company.find("span").text.strip()
    job_list[i][1] = comp_name

  return job_list


def get_wwr_jobs(term):
  job_list = []
  url = f"https://weworkremotely.com/remote-jobs/search?term={term}"
  html_doc = requests.get(url).text
  soup = BeautifulSoup(html_doc, "html.parser")
  list_soup = soup.find("ul")
  list_item = list_soup.find_all("li", {"class":"feature"})
  for info in list_item:
    title = info.find("span", {"class":"title"}).text
    company = info.find("span", {"class":"company"}).text
    link = "https://weworkremotely.com" + info.find("a").get("href")
    job_list.append([title, company, link])
  
  return job_list


def get_remo_jobs(term):
  job_list = []
  url = f"https://remoteok.io/remote-dev+{term}-jobs"
  html_doc = requests.get(url).text
  soup = BeautifulSoup(html_doc, "html.parser")
  tr_soup = soup.find_all("tr", {"class":"job"})
  for tr in tr_soup:
    title = tr.find("h2", {"itemprop":"title"}).text
    company = tr.get("data-company")
    link = "https://remoteok.io" + tr.get("data-href")
    job_list.append([title, company, link])

  return job_list


def write_csv(job_list):
  with open("jobs.csv", "w") as file:
    writer = csv.writer(file)
    for job_info in job_list:
      writer.writerow(job_info)
    file.close()

db= {}
app = Flask("RemoteJobs")

@app.route("/")
def home():
  return render_template("home.html")

@app.route("/search")
def search():
  global job_list
  search_key = request.args.get("term").lower()
  if search_key in db.keys():
    job_list = db[search_key]
  else:
    job_list = get_so_jobs(search_key) + get_wwr_jobs(search_key) + get_remo_jobs(search_key)
    db[search_key] = job_list

  return render_template("search.html", term=search_key, job_list=job_list, length=len(job_list))

@app.route("/jobs.csv")
def export_file():
  write_csv(job_list)
  return send_file("jobs.csv")



app.run()

 * Serving Flask app 'RemoteJobs'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [4]:
from mpl_toolkits.mplot3d import Axes3D
#from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)